In [1]:
#imports needed
import os
import torch
from torch import nn
import pandas as pd
import numpy as np


In [2]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

train = pd.read_csv('titanic_train.csv')
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)
train.drop('Cabin',axis=1,inplace=True)
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
train = pd.concat([train,sex,embark],axis=1)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train.drop('Survived',axis=1),train['Survived'], test_size=0.20,random_state=101)
X_train = torch.tensor(X_train.values).float()
X_test = torch.tensor(X_test.values).float()
Y_train = torch.tensor(Y_train.values).float()
Y_test = torch.tensor(Y_test.values).float()

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(9,5)
        self.fc2 = nn.Linear(5,3)
        self.fc3 = nn.Linear(3,1)
        

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
        

In [5]:
net = NeuralNetwork()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

for epoch in range(30): 
    for data in range(len(X_train)):
        optimizer.zero_grad()
        outputs = net(X_train[data])
        loss = criterion(outputs.squeeze(), Y_train[data])
        loss.backward()
        optimizer.step()

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
train_pred = net(X_train).detach().numpy()
for i in range(len(train_pred)):
    if train_pred[i] >= .5:
        train_pred[i] = 1
    else:
        train_pred[i] = 0
print('----------------------------------------')
print('Training sets stats')
print(classification_report(Y_train,train_pred))
print("Accuracy:", accuracy_score(Y_train, train_pred))


----------------------------------------
Training sets stats
              precision    recall  f1-score   support

         0.0       0.63      1.00      0.77       450
         1.0       0.00      0.00      0.00       262

    accuracy                           0.63       712
   macro avg       0.32      0.50      0.39       712
weighted avg       0.40      0.63      0.49       712

Accuracy: 0.6320224719101124


d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
test_pred = net(X_test).detach().numpy()
for i in range(len(test_pred)):
    if test_pred[i] >= .5:
        test_pred[i] = 1
    else:
        test_pred[i] = 0
print('----------------------------------------')
print('Test sets stats')
print(classification_report(Y_test,test_pred))
print("Accuracy:", accuracy_score(Y_test, test_pred))

----------------------------------------
Test sets stats
              precision    recall  f1-score   support

         0.0       0.55      1.00      0.71        99
         1.0       0.00      0.00      0.00        80

    accuracy                           0.55       179
   macro avg       0.28      0.50      0.36       179
weighted avg       0.31      0.55      0.39       179

Accuracy: 0.553072625698324


d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
